# Домашнее задание 3 [10 баллов] 
# До 30.04.18 23:59

Задание выполняется в группе (1-4 человека). В случае использования какого-либо строннего источника информации обязательно дайте на него ссылку (поскольку другие тоже могут на него наткнуться). Плагиат наказывается нулём баллов за задание и предвзятым отношением в будущем.

Не все части обязательны для выполнения, однако вы можете быть дополнительно оштрафованы за небрежное за выполнение одной или двух частей вместо четырех.

При возниконовении проблем с выполнением задания обращайтесь с вопросами к преподавателю. Поэтому настоятельно рекомендуется выполнять задание заранее, оставив запас времени на всевозможные технические проблемы. Если вы начали читать условие в последний вечер и не успели из-за проблем с установкой какой-либо библиотеки — это ваши проблемы.


Результат выполнения задания — это отчёт в формате html на основе Jupyter Notebook. Нормальный отчёт должен включать в себя:
* Краткую постановку задачи и формулировку задания
* Описание **минимума** необходимой теории и/или описание используемых инструментов - не стоит переписывать лекции или Википедию
* Подробный пошаговый рассказ о проделанной работе
* Аккуратно оформленные результаты
* **Внятные выводы** – не стоит относится к домашнему заданию как к последовательности сугубо технических шагов, а стоит относится скорее как к небольшому практическому исследованию, у которого есть своя цель и свое назначение.

Небрежное его оформление отчета существенно отразится на итоговой оценке. Весь код из отчёта должен быть воспроизводимым, если для этого нужны какие-то дополнительные действия, установленные модули и т.п. — всё это должно быть прописано в тексте в явном виде.

Сдача отчетов осуществляется через систему AnyTask.


## Использование архитектуры SENNA для определения части речи

Домашнее задание написано по мотивам работы R. Collobert:

**Collobert, Ronan, Jason Weston, Léon Bottou, Michael Karlen, Koray Kavukcuoglu, and Pavel Kuksa. "Natural language processing (almost) from scratch." Journal of Machine Learning Research 12, no. Aug (2011): 2493-2537.**

В этом домашнем задании вам предстоит самостоятельно разработать архитектуру SENNA для определения части речи. 
SENNA – это простая архитектура нейронной сети, позволяющая достигнуть state-of-the-art результатов в нескольких задачах обработки текстов.  

Использование SENNA для определения части речи предполагает, что задача определения части речи для данного слова формулируется как задача классификации: пусть в размеченном корпусе всего $|T|$ (= tagset) различных тегов частей речи, тогда каждое слово $w$ относится к одному из $T$ классов. Для каждого слова из обучающих данных формируется собственный вектор признаков. Нейронная сеть обучается по всем векторам признаков для слов из обучающего множества. 

Подход к решению задачи классификации представлен в оригинальной статье на рис. 1 (Figure 1: Window approach network). Он состоит из следующих шагов (раздел 3.3.1):
1. Каждое слово представляется эмбеддингом: $w_i \rightarrow LT_{w^i}$, размерность эмбеддинга - $d$;
2. Для каждого слова формируется окно длины $k$ из $(k-1)/2$ соседних слов слева от данного слова  и $(k-1)/2$ соседних слов справа от данного слова, $k$ – нечетное. 
3. Для каждого слова формируется вектор признаков, состоящий из конкатенированных эмбеддингов слов из левого окна, данного слова и слов из правого окна. Итоговая размерность вектора признаков – $d \times k$. Именно этот вектор подается на вход нейронной сети;
4. Обучается нейронная сеть, имеющая один скрытый слой с $n_h$ нейроннами и нелинейной функцией активации $\theta$;
5. На выходном слое нейронной сети решается задача классификации на |T| классов, то есть, определяется часть речи для каждого слова. 

Если для слова невозможно найти $(k-1)/2$ соседних слов слева от данного слова  и $(k-1)/2$ соседних слов справа от данного слова – используется padding.


### Данные
1. Открытый корпус: https://github.com/dialogue-evaluation/morphoRuEval-2017/blob/master/OpenCorpora_Texts.rar
2. Предобученные эмбеддинги Facebook: https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.ru.vec

### Часть 1 [2 балла] Подготовка данных
1. Прочитайте размеченные данные Открытого корпуса, используя nltk.corpus.reader.conll.ConllCorpusReader
2. Посчитайте количество предложений и число тегов частей речи;
3. Сформируйте тестовое и обучающее множество: первые 3/4 данных – обучающее множество;

Для каждого слова:
1. Определите его окно (слова слева и справа) размера $k$;
2. Сформируйте его вектор признаков.

In [1]:
from nltk.corpus.reader.conll import ConllCorpusReader
import numpy as np

In [2]:
reader = ConllCorpusReader('.', ['unamb_sent_14_6.conllu'], ('ignore', 'words', 'ignore', 'pos'))

Число предложений

In [3]:
sents = reader.tagged_sents()
N = len(sents)
print(N)

38508


Число тегов частей речи

In [4]:
all_pos = list(set(list(map(lambda x: x[1], reader.tagged_words()))))
NUM_CLASSES = len(all_pos)
print(NUM_CLASSES)

14


Делим предложения на обучающую и тестирующую часть

In [5]:
TRAIN = sents[:-N // 4]
TEST = sents[-N // 4:]

Функуции чтения эмбеддингов и подготовки датасета

In [10]:
def read_embeddings(max_words = -1):
    words = []
    embeddings = []
    file = open('wiki.ru.vec', 'r', encoding='UTF-8')
    file.readline()
    for i, line in enumerate(file):
        if max_words != -1 and i >= max_words:
            break
        line = line.strip()
        split_result = line.rsplit(maxsplit=300)
        word = split_result[0]
        embedding = np.array(split_result[1:],dtype=float)
        words.append(word)
        embeddings.append(embedding)
    return words, embeddings

In [11]:
def prepare_dataset(sents, word_to_idx, embeddings, pos_idx, k):
    p = (k - 1) // 2
    features = []
    labels = []
    for sent in sents:
        u = 0
        sent_embeddings = []
        unknown_embedding = np.zeros(300, dtype=float)
        sent_embeddings.extend([unknown_embedding] * p)
        for word, tag in sent:
            word = word.lower()
            if word in word_to_idx:
                word_embedding = embeddings[word_to_idx[word]]
            else:
                word_embedding = unknown_embedding
            sent_embeddings.append(word_embedding)
            labels.append(pos_idx[tag])
            u += 1
        sent_embeddings.extend([unknown_embedding] * p)
        for i in range(u):
            current = np.array(sent_embeddings[i:i + k])
            features.append(current.flatten())
    return np.array(features, dtype=float), np.array(labels,dtype=int)

In [12]:
def build_word_to_idx(words):
    res = {}
    for i, word in enumerate(words):
        res[word] = i
    return res

Считываем эмбеддинги (ограничиваемся 50000 самых популярных слов) и формируем датасет c окном ширины 3

In [13]:
words, embeddings = read_embeddings(50000)

In [14]:
word_to_idx = build_word_to_idx(words)
pos_idx = build_word_to_idx(all_pos)

In [15]:
k = 3
X_train, Y_train = prepare_dataset(TRAIN, word_to_idx, embeddings, pos_idx, k)
X_test, Y_test = prepare_dataset(TEST, word_to_idx, embeddings, pos_idx, k)

### Часть 2 [4 баллов] Архитектура нейронной сети

Архитектура нейронной сети состоит из следующих слов:
1. Входной слой: нейронная сеть получает на вход вектор признаков, состоящий из $k$ конкатенированных эмбеддингов;/
2. Скрытый слой: $n_h$ нейронов и нелинейная функция активации $\theta$;
3. Выходной слой:  $|T|$ нейронов для итоговой классификации.

Обучите нейронную сеть на обучающих данных.

In [16]:
import keras
from keras.layers import Input
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam

Using TensorFlow backend.


In [17]:
def build_model(n_h, k, act):
    model = Sequential()
    model.add(Dense(n_h, activation=act, input_shape=(k * 300,)))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    return model

In [18]:
model = build_model(128, k, 'relu')

In [19]:
model.compile(loss='sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [20]:
model.fit(X_train, Y_train, batch_size=64, epochs=1000, validation_split=0.1, \
          callbacks=[keras.callbacks.EarlyStopping('val_acc')])

Train on 315319 samples, validate on 35036 samples
Epoch 1/1000
315319/315319 [==============================] - 14s 44us/step - loss: 0.3842 - acc: 0.8710 - val_loss: 0.4123 - val_acc: 0.8567
Epoch 2/1000
315319/315319 [==============================] - 12s 38us/step - loss: 0.2871 - acc: 0.8995 - val_loss: 0.3877 - val_acc: 0.8666
Epoch 3/1000
315319/315319 [==============================] - 12s 38us/step - loss: 0.2590 - acc: 0.9087 - val_loss: 0.3851 - val_acc: 0.8674
Epoch 4/1000
315319/315319 [==============================] - 12s 38us/step - loss: 0.2410 - acc: 0.9144 - val_loss: 0.3789 - val_acc: 0.8705
Epoch 5/1000
315319/315319 [==============================] - 12s 38us/step - loss: 0.2272 - acc: 0.9189 - val_loss: 0.3928 - val_acc: 0.8705


### Часть 3 [1 балл] Оценка качества

Протестируйте нейронную сеть на тестовых данных. Используйте accuracy для оценки качества модели.

In [21]:
_, accuracy = model.evaluate(X_test, Y_test)
print(accuracy)

107228/107228 [==============================] - 4s 34us/step
0.890439064428


Получили качество порядка 90%

### Часть 4 [1 балл] Оптимизация гиперпарметров

В эксперименте участвуют следующие гиперпараметры:
* $k$ – размер окна;
* $n_h$ – число нейронов на скрытом слое;
* $\theta$ – вид функции активации.

Оцените их влияние на качество модели. Как увеличение окна или числа нейронов влияет на итоговый показатель качества? Зависит ли итоговый показатель качества от функции активации на скрытом слое? 

In [22]:
def fit_and_measure_quality(X_train, Y_train, X_test, Y_test, k, n_h, thetha):
    print('Building model...')
    model = build_model(n_h, k, thetha)
    model.compile(loss='sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.fit(X_train, Y_train, batch_size=64, epochs=1000, validation_split=0.1, \
          callbacks=[keras.callbacks.EarlyStopping('val_acc')])
    _, accuracy = model.evaluate(X_test, Y_test)
    return accuracy

In [23]:
def load_fit_and_measure_quality(k, n_h, thetha):
    print('Loading train dataset...')
    X_train, Y_train = prepare_dataset(TRAIN, word_to_idx, embeddings, pos_idx, k)
    print('Loading test dataset...')
    X_test, Y_test = prepare_dataset(TEST, word_to_idx, embeddings, pos_idx, k)
    return fit_and_measure_quality(X_train, Y_train, X_test, Y_test, k, n_h, thetha)

Оценим влияние ширины окна на показатель качества

In [24]:
for k in [1, 3, 5, 7]:
    print("Calculating for k=%d" % k)
    quality = load_fit_and_measure_quality(k, 128, 'relu')
    print("k=%d: Accuracy=%f" % (k, quality))

Calculating for k=1
Loading train dataset...
Loading test dataset...
Building model...
Train on 315319 samples, validate on 35036 samples
Epoch 1/1000
315319/315319 [==============================] - 12s 37us/step - loss: 0.4810 - acc: 0.8283 - val_loss: 0.5028 - val_acc: 0.8149
Epoch 2/1000
315319/315319 [==============================] - 11s 36us/step - loss: 0.4084 - acc: 0.8462 - val_loss: 0.4845 - val_acc: 0.8190
Epoch 3/1000
315319/315319 [==============================] - 11s 36us/step - loss: 0.3980 - acc: 0.8484 - val_loss: 0.4793 - val_acc: 0.8201
Epoch 4/1000
315319/315319 [==============================] - 11s 36us/step - loss: 0.3928 - acc: 0.8495 - val_loss: 0.4799 - val_acc: 0.8226
Epoch 5/1000
107228/107228 [==============================] - 3s 28us/step
k=1: Accuracy=0.841329
Calculating for k=3
Loading train dataset...
Loading test dataset...
Building model...
Train on 315319 samples, validate on 35036 samples
Epoch 1/1000
315319/315319 [==============================

Можно видеть, что использование окна шириной 3 заметно лучше чем единичное окно, однако дальнейшее увеличение не приводит к сильному росту качества. Тем не менее есть некоторый прирост при k=5, дальше качество не растет, так что используем k=5

Подберем оптимальное значение нейронов скрытого слоя. Обычно для подобного параметра должен быть некоторый минимум, т.к. при небольшом количестве наблюдается недообучение, а при большом напротив - переобучение.

In [25]:
k = 5
X_train, Y_train = prepare_dataset(TRAIN, word_to_idx, embeddings, pos_idx, k)
X_test, Y_test = prepare_dataset(TEST, word_to_idx, embeddings, pos_idx, k)

In [26]:
for n_h in [32, 64, 128, 256, 512]:
    print("Calculating for n_h=%d" % n_h)
    quality = fit_and_measure_quality(X_train, Y_train, X_test, Y_test, k, n_h, 'relu')
    print("n_h=%d: Accuracy=%f" % (n_h, quality))

Calculating for n_h=32
Building model...
Train on 315319 samples, validate on 35036 samples
Epoch 1/1000
315319/315319 [==============================] - 22s 69us/step - loss: 0.4302 - acc: 0.8608 - val_loss: 0.4276 - val_acc: 0.8538
Epoch 2/1000
315319/315319 [==============================] - 21s 67us/step - loss: 0.3033 - acc: 0.8959 - val_loss: 0.4096 - val_acc: 0.8613
Epoch 3/1000
315319/315319 [==============================] - 21s 67us/step - loss: 0.2797 - acc: 0.9034 - val_loss: 0.3916 - val_acc: 0.8669
Epoch 4/1000
315319/315319 [==============================] - 21s 67us/step - loss: 0.2643 - acc: 0.9089 - val_loss: 0.3866 - val_acc: 0.8679
Epoch 5/1000
315319/315319 [==============================] - 21s 67us/step - loss: 0.2520 - acc: 0.9124 - val_loss: 0.3907 - val_acc: 0.8686
Epoch 6/1000
107228/107228 [==============================] - 7s 69us/step
n_h=32: Accuracy=0.891931
Calculating for n_h=64
Building model...
Train on 315319 samples, validate on 35036 samples
Epoch

Судя по результатам оптимальное число нейронов должно лежать между 128 и 256, переберем некоторые значения в этом диапазоне.

In [27]:
for n_h in [120, 160, 200, 240]:
    print("Calculating for n_h=%d" % n_h)
    quality = fit_and_measure_quality(X_train, Y_train, X_test, Y_test, k, n_h, 'relu')
    print("n_h=%d: Accuracy=%f" % (n_h, quality))

Calculating for n_h=120
Building model...
Train on 315319 samples, validate on 35036 samples
Epoch 1/1000
315319/315319 [==============================] - 21s 66us/step - loss: 0.3854 - acc: 0.8721 - val_loss: 0.4129 - val_acc: 0.8596
Epoch 2/1000
315319/315319 [==============================] - 21s 65us/step - loss: 0.2756 - acc: 0.9048 - val_loss: 0.3911 - val_acc: 0.8671
Epoch 3/1000
315319/315319 [==============================] - 21s 65us/step - loss: 0.2394 - acc: 0.9161 - val_loss: 0.3844 - val_acc: 0.8703
Epoch 4/1000
315319/315319 [==============================] - 20s 64us/step - loss: 0.2134 - acc: 0.9252 - val_loss: 0.3931 - val_acc: 0.8721
Epoch 5/1000
315319/315319 [==============================] - 20s 64us/step - loss: 0.1920 - acc: 0.9322 - val_loss: 0.4070 - val_acc: 0.8724
Epoch 6/1000
315319/315319 [==============================] - 20s 64us/step - loss: 0.1747 - acc: 0.9379 - val_loss: 0.4182 - val_acc: 0.8737
Epoch 7/1000
107228/107228 [===========================

Остановимся на значении с максимальным accuracy: 160

Выберем функцию активации

In [28]:
n_h = 160
for nonlinearity in ['relu', 'elu', 'selu', 'sigmoid', 'tanh']:
    print("Calculating for %s" % nonlinearity)
    quality = fit_and_measure_quality(X_train, Y_train, X_test, Y_test, k, n_h, nonlinearity)
    print("function=%s: Accuracy=%f" % (nonlinearity, quality))

Calculating for relu
Building model...
Train on 315319 samples, validate on 35036 samples
Epoch 1/1000
315319/315319 [==============================] - 20s 65us/step - loss: 0.3784 - acc: 0.8738 - val_loss: 0.3987 - val_acc: 0.8639
Epoch 2/1000
315319/315319 [==============================] - 20s 64us/step - loss: 0.2694 - acc: 0.9068 - val_loss: 0.3799 - val_acc: 0.8717
Epoch 3/1000
315319/315319 [==============================] - 20s 63us/step - loss: 0.2313 - acc: 0.9191 - val_loss: 0.3843 - val_acc: 0.8721
Epoch 4/1000
315319/315319 [==============================] - 20s 63us/step - loss: 0.2032 - acc: 0.9288 - val_loss: 0.4060 - val_acc: 0.8722
Epoch 5/1000
315319/315319 [==============================] - 20s 64us/step - loss: 0.1802 - acc: 0.9367 - val_loss: 0.4128 - val_acc: 0.8732
Epoch 6/1000
107228/107228 [==============================] - 7s 66us/step
function=relu: Accuracy=0.894300
Calculating for elu
Building model...
Train on 315319 samples, validate on 35036 samples
Epo

Можно видеть, что выбор функции активации практически не влияет на результат. Остановимся на той, которая дает лучший результат - sigmoid.

Итоговые подобранные значения гиперпараметров
    - k=5
    - 160 неронов на скрытом слое
    - Нелинейность - sigmoid

### Часть 5 [2 балла] Анализ ошибок
1. Привидите примеры из тестового множества, на которых нейронная сеть ошибается. Объясните, почему возникают ошибки.
2. Протестируйте нейронную сеть на произвольном предложении (не из тестовых данных). Возникают ли ошибки? Почему?

Обучаем модель с оптимальными параметрами

In [29]:
model = build_model(n_h, k, 'sigmoid')
model.compile(loss='sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(X_train, Y_train, batch_size=64, epochs=1000, validation_split=0.1, \
      callbacks=[keras.callbacks.EarlyStopping('val_acc')])

Train on 315319 samples, validate on 35036 samples
Epoch 1/1000
315319/315319 [==============================] - 20s 65us/step - loss: 0.4352 - acc: 0.8603 - val_loss: 0.4183 - val_acc: 0.8577
Epoch 2/1000
315319/315319 [==============================] - 19s 62us/step - loss: 0.2963 - acc: 0.8989 - val_loss: 0.4002 - val_acc: 0.8628
Epoch 3/1000
315319/315319 [==============================] - 20s 63us/step - loss: 0.2678 - acc: 0.9072 - val_loss: 0.3870 - val_acc: 0.8697
Epoch 4/1000
315319/315319 [==============================] - 20s 63us/step - loss: 0.2460 - acc: 0.9146 - val_loss: 0.3768 - val_acc: 0.8731
Epoch 5/1000
315319/315319 [==============================] - 20s 63us/step - loss: 0.2280 - acc: 0.9212 - val_loss: 0.3760 - val_acc: 0.8739
Epoch 6/1000
315319/315319 [==============================] - 19s 62us/step - loss: 0.2116 - acc: 0.9261 - val_loss: 0.3774 - val_acc: 0.8734


Функция тестирования модели на размеченном предложении: возвращает долю ошибок в этом предложении и список строк с описаниями ошибок

In [71]:
def test_model_on_tagged_sentence(model, sent):
    features, labels = prepare_dataset([sent], word_to_idx, embeddings, pos_idx, k)
    predicted_labels = model.predict_classes(features)
    sent_predictions = list(map( lambda x: all_pos[x], predicted_labels))
    sent_accuracy = sum(predicted_labels == labels) / len(labels)
    errors = []
    for tagged, prediction in zip(sent, sent_predictions):
        word, actual = tagged
        if actual != prediction:
            errors.append("Word %s: Expected %s Prediction %s" % (word, actual, prediction))
    return errors

In [96]:
def print_sent(tagged_sent):
    words = []
    for word, _ in tagged_sent:
        words.append(word)
    print(' '.join(words))

Отберем достаточно длинные предложения (не менее 7 слов), на которых модель допускает не менее 4 ошибок

In [97]:
results = []
for sent in TEST:
    if len(sent) < 7:
        continue
    errors = test_model_on_tagged_sentence(model, sent)
    if len(errors) >= 4:
        results.append((sent, errors))

Посмотрим на результаты 3 таких предложений

In [99]:
from random import shuffle
shuffle(results)
for sent, errors in results[:3]:
    print_sent(sent)
    for error in errors:
        print(error)
    print()

Кажется сомнительным , что те « приёмы » в работе , которые студенты используют для обхода ошибок , наличествующих в используемом СГА проприетарном ПО , соответствуют миссии СГА — предоставлению образования высшего качества .
Word используемом: Expected ADJ Prediction NOUN
Word СГА: Expected PROPN Prediction NOUN
Word проприетарном: Expected ADJ Prediction VERB
Word ПО: Expected NOUN Prediction ADP
Word СГА: Expected PROPN Prediction X
Word предоставлению: Expected NOUN Prediction ADJ

Из-за мелких размеров постройки / проходы гнезда незаметны , обитают в естественных полостях почвы , фураж осуществляют в почве и листовом опаде .
Word Из-за: Expected ADP Prediction NUM
Word незаметны: Expected ADJ Prediction NOUN
Word полостях: Expected NOUN Prediction ADJ
Word фураж: Expected NOUN Prediction ADJ
Word опаде: Expected X Prediction NOUN

Традиция признаёт Цзо чжуань комментарием к летописи Чунь цю , однако трактовка и подробности событий не совпадают с летописью и другими комментариями .

Во всех 3 предложениях ошибки возникают в основном на словах, которых нет в 50000 самых популярных слов из таблицы эмбеддингов - иногда из-за редкости самого слова, а иногда из-за редкости конкретной словоформы, а информации из левого и правого контекста недостаточно для однозначного определения части речи. Исключением является разве что слово "комментарием" из 3-его предложения, в нем судя по всему ошибка связана с тем, что в левом контексте оба слова с неизвестным эмбеддингом.

Протестируем модель на произвольном предложении не из выборки